In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import time
import statistics
import sklearn
%matplotlib inline
from datetime import date
from scipy import stats as stats
from sklearn import preprocessing
from sklearn import model_selection
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler

In [2]:
# Accesses relevant files and uses col_list to access only needed columns
df_ADMISSIONS = pd.read_csv("/Users/alex/ADMISSIONS.csv")
col_list = ["SUBJECT_ID", "DOB", "GENDER", "EXPIRE_FLAG"]
df_PATIENTS = pd.read_csv("/Users/alex/PATIENTS.csv", usecols=col_list)

#Merges PATIENTS and ADMISSIONS on SUBJCT_ID
df_ADMISSIONS = pd.merge(df_ADMISSIONS, df_PATIENTS,on='SUBJECT_ID',how='outer',indicator=True)

#Finds the difference between two times in hours, adding them to the list time_in
date_mask = "%Y-%m-%d %H:%M:%S"
df_ADMISSIONS['ADMITTIME'] = pd.to_datetime(df_ADMISSIONS['ADMITTIME'], format= date_mask)
df_ADMISSIONS['DISCHTIME'] = pd.to_datetime(df_ADMISSIONS['DISCHTIME'], format= date_mask)
df_ADMISSIONS['DOB'] = pd.to_datetime(df_ADMISSIONS['DOB'], format= date_mask)
df_ADMISSIONS['LOS_hr'] = round(((df_ADMISSIONS['DISCHTIME'] - df_ADMISSIONS['ADMITTIME']).dt.total_seconds() / 3600))

#Finds the difference between two times in years, adding them to the list age
age = []
for i in range(0, len(df_ADMISSIONS)):
    datetime1 = pd.to_datetime(df_ADMISSIONS['DOB'][i], format = date_mask)
    datetime2 = pd.to_datetime(df_ADMISSIONS['ADMITTIME'][i], format = date_mask)
    days = datetime2 - datetime1
    years_old = int(round(days.days/(366)))
    age.append(years_old)
#Appends age to ADMISSIONS under column 'AGE'
df_ADMISSIONS['AGE'] = age


#Replaces patients over the age of 140 with the median age this group truly was of 91.
for i in range(0, len(df_ADMISSIONS)):
    if df_ADMISSIONS.AGE[i] > 140:
        df_ADMISSIONS["AGE"].replace(df_ADMISSIONS.AGE[i], 91, inplace = True)

In [3]:
# Drops Patient Records from ADMISSIONS based on exclusion criteria in paper.
df_ADMISSIONS = df_ADMISSIONS.drop(df_ADMISSIONS[df_ADMISSIONS.EXPIRE_FLAG == 1].index)
df_ADMISSIONS = df_ADMISSIONS.drop_duplicates(subset='SUBJECT_ID', keep='first')
df_ADMISSIONS = df_ADMISSIONS.drop(df_ADMISSIONS[df_ADMISSIONS.HAS_CHARTEVENTS_DATA == 0].index)
df_ADMISSIONS = df_ADMISSIONS.drop(df_ADMISSIONS[df_ADMISSIONS.AGE < 15].index)
df_ADMISSIONS = df_ADMISSIONS.drop(df_ADMISSIONS[df_ADMISSIONS.LOS_hr < 0.1].index)

In [4]:
#Opens columns that appear in 'col_list' from CHARTEVENTS and OUTPUTEVENTS
col_list = ["HADM_ID", "ITEMID", 'CHARTTIME', 'VALUE']
df_OUTPUT = pd.read_csv("/Users/alex/OUTPUTEVENTS.csv", usecols=col_list, chunksize = 1000000)
df_EVENTS = pd.read_csv("/Users/alex/CHARTEVENTS.csv", usecols=col_list, chunksize = 1000000)

#Used to pair events with patient stays
df_HADM = df_ADMISSIONS[['HADM_ID']]

#Goes through chunks merging events that match HADM_ID of patients to df_HADM
def preprocessing(chunk, df, lst):
    df = pd.DataFrame(chunk)
    common = df_HADM.merge(df,on=['HADM_ID','HADM_ID'])
    df_HADM[(~df_HADM.HADM_ID.isin(common.HADM_ID))&(~df_HADM.HADM_ID.isin(common.HADM_ID))]
    common = pd.DataFrame(common)
    lst.append(common) 


output_lst = []
for chunk in df_OUTPUT:
    preprocessing(chunk, df_OUTPUT, output_lst)  
output = pd.concat(output_lst)


events_lst = []
for chunk in df_EVENTS:
    preprocessing(chunk, df_EVENTS, events_lst)  
events = pd.concat(events_lst)

/Users/alex/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
#This function searches CHARTEVENTS for the ITEMIDs given in each list and forms them into a df named by the vital 
#it contains
def sort_vitals(Vital_name,Vital, maxval, minval):
    boolean_series = events.ITEMID.isin(Vital)
    filtered_df = events[boolean_series].copy()
    filtered_df['VALUE'] = pd.to_numeric(filtered_df['VALUE'], errors='coerce')
    filtered_df.sort_values(by=['HADM_ID', 'CHARTTIME'])
    filtered_df.fillna(method='ffill', inplace=True)
    filtered_df['ITEM_NAME'] = Vital_name
    filtered_df = filtered_df.drop(filtered_df[filtered_df.VALUE > maxval].index)
    Vital = pd.DataFrame(filtered_df.drop(filtered_df[filtered_df.VALUE < minval].index))
    return Vital


systolicBP = [227243, 224167, 220179, 225309, 220050, 442, 455, 6701, 51]
SystolicBP = sort_vitals('systolicBP',systolicBP, 450, 0)

Potassium =[227464, 829, 1535]
Potassium = sort_vitals('Potassium',Potassium, 1000, 0)

white_blood_cell = [220546, 861, 1127, 1542]
White_Blood_Cell = sort_vitals('White_blood_cell',white_blood_cell, 1000, 0)

sodium = [220645, 226534, 837, 1536]
Sodium = sort_vitals('sodium', sodium, 10000, 0)

bicarbonate = [227443]
Bicarbonate = sort_vitals('bicarbonate', bicarbonate, 1000, 0)

glasgow_coma = [198]
Glasgow_Coma = sort_vitals('glasgow', glasgow_coma, 1000, 0)

birillium = [225690, 848, 1538, 225651, 803]
Birillium = sort_vitals('birillium', birillium, 10000, 0)

Blood_Urea_Nitrogen = [225624, 781, 1162]
BloodUreaNitrogen = sort_vitals('BUN', Blood_Urea_Nitrogen, 50000, 0)

mech_vent = [223835, 223835, 3420, 3422, 190]
Mech_Vent = sort_vitals('mech_vent', mech_vent, 50000, 0)

#These vitals required special manipulation so have their own code.
HeartRate = [220045, 211]
boolean_series = events.ITEMID.isin(HeartRate)
filtered_df = events[boolean_series].copy()
filtered_df['VALUE'] = pd.to_numeric(filtered_df['VALUE'], errors='coerce')
filtered_df.sort_values(by=['HADM_ID', 'CHARTTIME'])
filtered_df.fillna(method='ffill', inplace=True)
filtered_df['ITEM_NAME'] = 'HeartRate'
filtered_df.VALUE = pd.to_numeric(filtered_df.VALUE)
HeartRate = pd.DataFrame(filtered_df.drop(filtered_df[filtered_df.VALUE > 480].index))

temperature = [223761, 223762, 676, 677, 678, 679]
boolean_series = events.ITEMID.isin(temperature)
filtered_df = events[boolean_series].copy()
filtered_df['VALUE'] = pd.to_numeric(filtered_df['VALUE'], errors='coerce')
filtered_df.sort_values(by=['HADM_ID', 'CHARTTIME'])
filtered_df.fillna(method='ffill', inplace=True)
filtered_df['ITEM_NAME'] = 'temperature'
filtered_df['VALUE'] = np.where(filtered_df['ITEMID'] == 223761, ((filtered_df['VALUE'] - 32) * 5/9), filtered_df['VALUE'])
filtered_df = filtered_df.drop(filtered_df[filtered_df.VALUE > 47].index)
Temperature = pd.DataFrame(filtered_df.drop(filtered_df[filtered_df.VALUE < 10].index))

urine = [40055, 43175, 40069, 40094, 40715, 40473, 40085, 40057, 40056, 40405, 40428, 40086,
         40096, 40651, 226559, 226560, 226561, 226584, 226563, 226564, 226565, 226567, 226557, 
         226558, 227488, 227489]
boolean_series = output.ITEMID.isin(urine)
filtered_df = output[boolean_series].copy()
filtered_df['VALUE'] = pd.to_numeric(filtered_df['VALUE'], errors='coerce')
filtered_df.sort_values(by=['HADM_ID', 'CHARTTIME'])
filtered_df.fillna(method='ffill', inplace=True)
filtered_df['ITEM_NAME'] = 'Urine'
Urine = filtered_df.drop(filtered_df[filtered_df.VALUE > 50000].index)
Urine = pd.DataFrame(filtered_df.drop(filtered_df[filtered_df.VALUE < 0].index))

#Urine was only available from OUTPUT not EVENTS so this searches OUTPUT instead of EVENTS
col_list = ["ITEMID", 'VALUEUOM']
df_VALUEUOM = pd.read_csv("/Users/alex/OUTPUTEVENTS.csv", usecols=col_list, nrows = 10000000)
df_VALUEUOM = df_VALUEUOM.drop_duplicates(subset='ITEMID', keep='first')
show = df_VALUEUOM.loc[df_VALUEUOM['ITEMID'].isin(urine)]

In [7]:
#Fetches relevant columns from DIAGNOSES list
diagnose_list = ['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE']
df_DIAGNOSES = pd.read_csv("/Users/alex/DIAGNOSES_ICD.csv", usecols = diagnose_list)
df_HADM = df_HADM.reset_index(drop=True)

#Keeps diagnoses of patients who passed the elimination criteria 
common = df_HADM.merge(df_DIAGNOSES,on=['HADM_ID','HADM_ID'])
df_HADM[(~df_HADM.HADM_ID.isin(common.HADM_ID))&(~df_HADM.HADM_ID.isin(common.HADM_ID))]
df_DIAGNOSES = pd.DataFrame(common)

#Puts ICD9 codes into a list
ICD_9 = df_DIAGNOSES['ICD9_CODE'].tolist()
E = []

#Tranforms them into the first 3 characters if a number of beginning with V and first 4 if beginning with E
for i in range(0, len(ICD_9)):
    if ICD_9[i][0] == 'E':
        E.append(ICD_9[i][0:4])
    elif ICD_9[i][0] == 'V':
        E.append(ICD_9[i][0:3])
    else:
        E.append(ICD_9[i][0:3])
    

ICD_9 = E

#creates new dataframe to hold codes
df_ICD9CODES = df_DIAGNOSES[['ICD9_CODE', 'HADM_ID']]
df_ICD9CODES['ICD_CAT'] = ICD_9


#Dicitionary with the 18 categories codes will be sorted into
ICDDict = {
1 : 'infectious and parasitic',
2 : 'neoplasms',
3 : 'endocrine, metabolic and immunity',
4 : 'blood and blood-forming organs',
5 : 'mental',
6 : 'nervous',
7 : 'circulatory',
8 : 'respiratory',
9 : 'digestive',
10 : 'genitourinary',
11 : 'pregnancy, childbirth, and puerperium',
12 : 'skin and subcutaneous tissue',
13 : 'musculoskeletal',
14 : 'congenital',
15 : 'perinatal period',
16 : 'ill-defined',
17 : 'injury and poisoning',
18 : 'external causes of injury and supplemental classification'
}

NAME = []
CAT = []
#Goes through all ICD9 codes and appends the dictionary value for the ICD9 codes to list 'CAT'
for i in range(0, 236104):
    if ICD_9[i][0:1] == 'E' or ICD_9[i][0:1] == 'V':
        CAT.append(18)
    elif int(ICD_9[i]) <= 139:
        CAT.append(1)
    elif int(ICD_9[i]) > 139 and int(ICD_9[i]) <= 239:
        CAT.append(2)
    elif int(ICD_9[i]) > 239 and int(ICD_9[i]) <= 279:
        CAT.append(3)
    elif int(ICD_9[i]) > 279 and int(ICD_9[i]) <= 289:
        CAT.append(4)
    elif int(ICD_9[i]) > 289 and int(ICD_9[i]) <= 319:
        CAT.append(5)
    elif int(ICD_9[i]) > 319 and int(ICD_9[i]) <= 389:
        CAT.append(6)
    elif int(ICD_9[i]) > 389 and int(ICD_9[i]) <= 459:
        CAT.append(7)
    elif int(ICD_9[i]) > 460 and int(ICD_9[i]) <= 519:
        CAT.append(8)
    elif int(ICD_9[i]) > 519 and int(ICD_9[i]) <= 579:
        CAT.append(9)
    elif int(ICD_9[i]) > 579 and int(ICD_9[i]) <= 629:
        CAT.append(10)
    elif int(ICD_9[i]) > 629 and int(ICD_9[i]) <= 679:
        CAT.append(11)
    elif int(ICD_9[i]) > 679 and int(ICD_9[i]) <= 709:
        CAT.append(12)
    elif int(ICD_9[i]) > 709 and int(ICD_9[i]) <= 739:
        CAT.append(13)
    elif int(ICD_9[i]) > 739 and int(ICD_9[i]) <= 759:
        CAT.append(14)
    elif int(ICD_9[i]) > 759 and int(ICD_9[i]) <= 779:
        CAT.append(15)
    elif int(ICD_9[i]) > 779 and int(ICD_9[i]) <= 799:
        CAT.append(16)
    elif int(ICD_9[i]) > 799 and int(ICD_9[i]) <= 999:
        CAT.append(17)

#Creates new column for sorted codes and replaces the existing name with the name matched in the dictionary
df_ICD9CODES['CATEGORY_ICD9'] = CAT
df_ICD9CODES['ICD9_NAME'] = df_ICD9CODES['CATEGORY_ICD9'].replace(ICDDict)

#Groups 'df_ICD9CODES' by HADM_ID and resets index
ICD9HADMSort = df_ICD9CODES.groupby('HADM_ID')['ICD9_NAME'].apply(list).reset_index()




In [12]:
#Creates dummy variables for ICD9 Codes if you want to stakc, add .stack() after apply
ICD9_Dummy = pd.get_dummies(ICD9HADMSort['ICD9_NAME'].apply(pd.Series).stack(), drop_first=False).sum(level=0)

#Pairs them back with the inital data frame
ICD9_Dummy = ICD9_Dummy.join(ICD9HADMSort['HADM_ID'], how="outer")

In [14]:
#Pairs them back with the inital data frame
ICD9_Dummy = ICD9_Dummy.join(ICD9HADMSort['HADM_ID'], how="outer")

In [15]:
#List of all vital features used in SAPS df
vitals_lst = [Mech_Vent, Urine, BloodUreaNitrogen, Birillium, Glasgow_Coma, Bicarbonate, Sodium, White_Blood_Cell,
      Temperature, Potassium, SystolicBP, HeartRate]

#Brings all vitals together sorting them by chart time 
SAPSTime = pd.concat(vitals_lst, sort = False)
SAPSTime = SAPSTime.sort_values('CHARTTIME', ascending = False)

#Merges SAPSTime with ICD9_Dummy removing instances where HADM_ID does not match to anything
common = SAPSTime.merge(ICD9_Dummy,on=['HADM_ID','HADM_ID'])
SAPSTime[(~SAPSTime.HADM_ID.isin(common.HADM_ID))&(~SAPSTime.HADM_ID.isin(common.HADM_ID))]
SAPSICD9 = pd.DataFrame(common)

#New data frame with relevant demogrpahic columns
df_SAPSDemo = df_ADMISSIONS[['AGE', 'HADM_ID', 'ADMISSION_TYPE', 'LOS_hr', 'ADMITTIME']]

#Dummies 'ADMISSION_TYPE'
admission_dummy = pd.get_dummies(df_SAPSDemo['ADMISSION_TYPE'].apply(pd.Series).stack(), drop_first=True).sum(level = 0)
admission_dummy = admission_dummy.join(df_SAPSDemo['HADM_ID'], how = "outer")
common = df_SAPSDemo.merge(admission_dummy,on=['HADM_ID','HADM_ID'])

#Adds admission dummy to SAPSDemo removing the old addmission type
df_SAPSDemo[(~df_SAPSDemo.HADM_ID.isin(common.HADM_ID))&(~df_SAPSDemo.HADM_ID.isin(common.HADM_ID))]
df_SAPSDemo = pd.DataFrame(common)
df_SAPSDemo = df_SAPSDemo.drop(['ADMISSION_TYPE'], axis = 1)

#Bins ages into 16 categories, dummies them and then adds them to SAPSDemo removing the old age column
pd.cut(df_SAPSDemo.AGE, 10)
bins = [14, 19, 24, 29, 34, 39, 44, 49, 54, 59, 64, 69, 74, 79, 84, 89, 99]
labels =['1','2','3','4', '5', '6' ,'7' ,'8' ,'9', '10', '11', '12', '13', '14', '15', '16']
df_SAPSDemo['AGE BIN'] = pd.cut(df_SAPSDemo['AGE'], bins,labels=labels)
df_SAPSDemo = df_SAPSDemo.drop(['AGE'], axis = 1)
age_item = pd.get_dummies(df_SAPSDemo['AGE BIN'], prefix='Age Bin', drop_first=True)
df_SAPSDemo = df_SAPSDemo.join(age_item)
SAPS = df_SAPSDemo.drop(['AGE BIN'], axis = 1)

#Merges processed ICD9 codes and processed demographics to a final table 'SAPSICD9'
common = SAPSICD9.merge(df_SAPSDemo,on=['HADM_ID','HADM_ID'])
SAPSICD9[(~SAPSICD9.HADM_ID.isin(common.HADM_ID))&(~SAPSICD9.HADM_ID.isin(common.HADM_ID))]
SAPSICD9 = pd.DataFrame(common)
SAPS = SAPSICD9.drop(['AGE BIN'], axis = 1)
DischargeTime = df_ADMISSIONS[['DISCHTIME', 'HADM_ID']]
SAPSICD9 = SAPSICD9.merge(DischargeTime,on=['HADM_ID','HADM_ID'])

In [17]:
#Works out hours in, hours left
SAPSICD9['ADMITTIME'] = pd.to_datetime(SAPSICD9['ADMITTIME'], format= date_mask)
SAPSICD9['CHARTTIME'] = pd.to_datetime(SAPSICD9['CHARTTIME'], format= date_mask)
SAPSICD9['DISCHTIME'] = pd.to_datetime(SAPSICD9['DISCHTIME'], format= date_mask)
SAPSICD9['HOURS_IN'] = round(((SAPSICD9['CHARTTIME'] - SAPSICD9['ADMITTIME']).dt.total_seconds() / 3600))
SAPSICD9['HOURS_LEFT'] = round(((SAPSICD9['DISCHTIME'] - SAPSICD9['CHARTTIME']).dt.total_seconds() / 3600))
SAPSICD9['SHIFTS_LEFT'] = round((SAPSICD9['HOURS_LEFT'] / 8))

,HADM_ID,ITEMID,CHARTTIME,VALUE,ITEM_NAME,blood and blood-forming organs,circulatory,congenital,digestive,"endocrine, metabolic and immunity",...,Age Bin_11,Age Bin_12,Age Bin_13,Age Bin_14,Age Bin_15,Age Bin_16,DISCHTIME,HOURS_IN,HOURS_LEFT,SHIFTS_LEFT
8044958,118464,220045,2100-06-08 00:18:00,68.000000,HeartRate,2,2,0,1,2,...,0,0,0,0,0,0,2100-06-09 17:09:00,4.0,41.0,5.0
8044957,118464,223761,2100-06-08 00:19:00,36.722222,temperature,2,2,0,1,2,...,0,0,0,0,0,0,2100-06-09 17:09:00,4.0,41.0,5.0
8044956,118464,220179,2100-06-08 00:22:00,101.000000,systolicBP,2,2,0,1,2,...,0,0,0,0,0,0,2100-06-09 17:09:00,4.0,41.0,5.0
8044951,118464,220645,2100-06-08 00:34:00,141.000000,sodium,2,2,0,1,2,...,0,0,0,0,0,0,2100-06-09 17:09:00,5.0,41.0,5.0
8044952,118464,227443,2100-06-08 00:34:00,24.000000,bicarbonate,2,2,0,1,2,...,0,0,0,0,0,0,2100-06-09 17:09:00,5.0,41.0,5.0
8044955,118464,225624,2100-06-08 00:34:00,7.000000,BUN,2,2,0,1,2,...,0,0,0,0,0,0,2100-06-09 17:09:00,5.0,41.0,5.0
8044954,118464,220546,2100-06-08 00:34:00,2.000000,White_blood_cell,2,2,0,1,2,...,0,0,0,0,0,0,2100-06-09 17:09:00,5.0,41.0,5.0
8044953,118464,225690,2100-06-08 00:34:00,0.100000,birillium,2,2,0,1,2,...,0,0,0,0,0,0,2100-06-09 17:09:00,5.0,41.0,5.0
8044949,118464,220179,2100-06-08 01:00:00,99.000000,systolicBP,2,2,0,1,2,...,0,0,0,0,0,0,2100-06-09 17:09:00,5.0,40.0,5.0
8044950,118464,220045,2100-06-08 01:00:00,59.000000,HeartRate,2,2,0,1,2,...,0,0,0,0,0,0,2100-06-09 17:09:00,5.0,40.0,5.0


In [19]:
#Sorts df by time and rounds readings to hours
SAPSICD9['CHARTTIME'] = pd.to_datetime(SAPSICD9['CHARTTIME']) 
SAPSICD9['CHARTTIME'] = SAPSICD9['CHARTTIME'].dt.round('H')
SAPSICD9 = SAPSICD9.sort_values(by=['CHARTTIME'], ascending = True)

In [20]:
SAPSICD9.to_csv('SAPSICD9.csv')